# Show a passage

accented+vocalized, vocalized, consonantal

In [1]:
import sys,os,re
import collections
from IPython.display import HTML, display_pretty, display_html

import laf
from laf.fabric import LafFabric
from etcbc.lib import Transcription
from etcbc.preprocess import prep

fabric = LafFabric()

source = 'etcbc'
versions = ('4b', '4c')

  0.00s This is LAF-Fabric 4.8.3
API reference: http://laf-fabric.readthedocs.org/en/latest/texts/API-reference.html
Feature doc: https://shebanq.ancient-data.org/static/docs/featuredoc/texts/welcome.html



In [2]:
FF = {}
MSG = {}
LL = {}
for version in ('4b', '4c'):
    API = fabric.load(source+version, '--', 'passage', {
        "xmlids": {"node": False, "edge": False},
        "features": ('''
            otype
            g_cons g_word g_cons_utf8 g_word_utf8 g_word trailer_utf8
            book chapter verse label
        ''',''),
        "prepare": prep(select={'L'}),
        "primary": False,
    }, verbose='NORMAL')
    FF[version] = API['F']
    MSG[version] = API['msg']
    LL[version] = API['L']


  0.00s LOADING API: please wait ... 
  0.00s USING main: etcbc4b DATA COMPILED AT: 2015-11-02T15-08-56
  2.28s LOGFILE=/Users/dirk/laf/laf-fabric-output/etcbc4b/passage/__log__passage.txt
  2.29s INFO: LOADING PREPARED data: please wait ... 
  2.29s prep prep: G.node_sort
  2.34s prep prep: G.node_sort_inv
  2.80s prep prep: L.node_up
  5.37s prep prep: L.node_down
    10s prep prep: V.verses
    10s prep prep: V.books_la
    10s ETCBC reference: http://laf-fabric.readthedocs.org/en/latest/texts/ETCBC-reference.html
    11s INFO: LOADED PREPARED data
    11s INFO: DATA LOADED FROM SOURCE etcbc4b AND ANNOX  FOR TASK passage AT 2016-11-09T19-10-06
  0.00s LOADING API: please wait ... 
  0.00s BEGIN COMPILE m: etcbc4c
  0.00s LOGFILE=/Users/dirk/laf/laf-fabric-data/etcbc4c/bin/__log__compile__.txt
  0.00s PARSING ANNOTATION FILES
  0.08s INFO: parsing etcbc4c_regions.xml
  5.37s INFO: parsing etcbc4c_monads.xml
    28s INFO: parsing etcbc4c_lingo.xml
 1m 22s INFO: parsing etcbc4c_section

In [3]:
verses = {}
for version in versions:
    msg = MSG[version]
    F = FF[version]
    msg("{}: Making a mapping between a passage specification and a verse node".format(version))
    versesv = collections.defaultdict(lambda: collections.defaultdict(lambda: {}))
    for vn in F.otype.s('verse'):
        bk = F.book.v(vn)
        ch = int(F.chapter.v(vn))
        vs = int(F.verse.v(vn))
        versesv[bk][ch][vs] = vn
    verses[version] = versesv
    msg('Done')

  6.65s 4b: Making a mapping between a passage specification and a verse node
  8.34s Done
  8.34s 4c: Making a mapping between a passage specification and a verse node
  9.62s Done


In [4]:
HTML('''
<style type="text/css">
td.ht {
    font-family: Ezra SIL, SBL Hebrew, Verdana, sans-serif;
    font-size: x-large;
    line-height: 1.7;
    text-align: right;
    direction: rtl;
}
td.et {
    font-family: Verdana, sans-serif;
    font-size: medium;
    line-height: 1.2;
    text-align: left;
    direction: ltr;
}
td.vl {
    font-family: Verdana, Arial, sans-serif;
    font-size: small;
    text-align: right;
    vertical-align: top;
    color: #aaaaaa;
    width: 5%;
    direction: ltr;
}
</style>
''')

In [5]:
accent_pat = re.compile('[*0-9]')
tr = Transcription()

def print_verse(bk, ch, vs, vowels=True, accents=True):
    rows = {}
    for version in versions:
        F = FF[version]
        L = LL[version]
        label = '{} {}:{}'.format(bk, ch, vs)
        vn = verses[version][bk][ch][vs]
        treps = []
        trepes = []
        for w in L.d('word', vn):
            if not vowels:
                trep = '{}{}'.format(F.g_cons_utf8.v(w), F.trailer_utf8.v(w))
                trepe = F.g_cons.v(w)
            else:
                trep = '{}{}'.format(F.g_word_utf8.v(w), F.trailer_utf8.v(w))
                trepe = F.g_word.v(w)
                if not accents:
                    trep = Transcription.to_hebrew(accent_pat.sub('', tr.from_hebrew(trep)))
            treps.append(trep)
            trepes.append(trepe)
        text = ''.join(treps)
        texte = ' '.join(trepes)
        rows[version] = '''
    <tr><td class="vl">{}</td><td class="ht">{}</td></tr>
    <tr><td class="vl">{}</td><td class="et">{}</td></tr>
        '''.format(version, text, label, texte)
    return '''
<table>
    {}
</table>'''.format('\n'.join(rows[version] for version in versions))

pc = lambda bk, ch, vs: print_verse(bk, ch, vs, vowels=False, accents=False)    # no vowels, no accents
pv = lambda bk, ch, vs: print_verse(bk, ch, vs, vowels=True,  accents=False)    # vowels, no accents
pa = lambda bk, ch, vs: print_verse(bk, ch, vs, vowels=True,  accents=True)     # vowels and accents

In [6]:
HTML(pc('Esther', 3, 4))

4b,ויהי באמרם אליו יום ויום ולא שׁמע אליהם ויגידו להמן לראות היעמדו דברי מרדכי כי־הגיד להם אשׁר־הוא יהודי׃
Esther 3:4,W JHJ B >MRM >LJW JWM W JWM W L> CM< >LJHM W JGJDW L HMN L R>WT H JCR HW> JHWDJ
4c,ויהי באמרם אליו יום ויום ולא שׁמע אליהם ויגידו להמן לראות היעמדו דברי מרדכי כי־הגיד להם אשׁר־הוא יהודי׃
Esther 3:4,W JHJ B >MRM >LJW JWM W JWM W L> CM< >LJHM W JGJDW L HMN L R>WT H JCR HW> JHWDJ


In [7]:
HTML(pv('Esther', 3, 4))

4b,וַיְהִי באמרם אֵלָיו יֹום וָיֹום וְלֹא שָׁמַע אֲלֵיהֶם וַיַּגִּידוּ לְהָמָן לִרְאֹות הֲיַעַמְדוּ דִּבְרֵי מָרְדֳּכַי כִּי־הִגִּיד לָהֶם אֲשֶׁר־הוּא יְהוּדִי
Esther 3:4,WA- J:HI81J *B- *>MRM >;L@JW03 JO74WM W@- JO80WM W:- LO71> C@MA73< >:AL;JHE92M WA- J.AG.I74JDW. L:- H@M@81N LI- R:>OWT03 H:A- JA75:ACER& H71W.> J:HW.DI75J00
4c,וַיְהִי באמרם אֵלָיו יֹום וָיֹום וְלֹא שָׁמַע אֲלֵיהֶם וַיַּגִּידוּ לְהָמָן לִרְאֹות הֲיַעַמְדוּ דִּבְרֵי מָרְדֳּכַי כִּי־הִגִּיד לָהֶם אֲשֶׁר־הוּא יְהוּדִי
Esther 3:4,WA- J:HI81J *B- *>MRM >;L@JW03 JO74WM W@- JO80WM W:- LO71> C@MA73< >:AL;JHE92M WA- J.AG.I74JDW. L:- H@M@81N LI- R:>OWT03 H:A- JA45:ACER H71W.> J:HW.DI75J


In [8]:
HTML(pa('Esther', 3, 4))

4b,וַיְהִ֗י ֯ב֯אמרם אֵלָיו֙ יֹ֣ום וָיֹ֔ום וְלֹ֥א שָׁמַ֖ע אֲלֵיהֶ֑ם וַיַּגִּ֣ידוּ לְהָמָ֗ן לִרְאֹות֙ הֲיַֽעַמְדוּ֙ דִּבְרֵ֣י מָרְדֳּכַ֔י כִּֽי־הִגִּ֥יד לָהֶ֖ם אֲשֶׁר־ה֥וּא יְהוּדִֽי׃
Esther 3:4,WA- J:HI81J *B- *>MRM >;L@JW03 JO74WM W@- JO80WM W:- LO71> C@MA73< >:AL;JHE92M WA- J.AG.I74JDW. L:- H@M@81N LI- R:>OWT03 H:A- JA75:ACER& H71W.> J:HW.DI75J00
4c,וַיְהִ֗י באמרם אֵלָיו֙ יֹ֣ום וָיֹ֔ום וְלֹ֥א שָׁמַ֖ע אֲלֵיהֶ֑ם וַיַּגִּ֣ידוּ לְהָמָ֗ן לִרְאֹות֙ הֲיַֽעַמְדוּ֙ דִּבְרֵ֣י מָרְדֳּכַ֔י כִּֽי־הִגִּ֥יד לָהֶ֖ם אֲשֶׁר־ה֥וּא יְהוּדִֽי׃
Esther 3:4,WA- J:HI81J *B- *>MRM >;L@JW03 JO74WM W@- JO80WM W:- LO71> C@MA73< >:AL;JHE92M WA- J.AG.I74JDW. L:- H@M@81N LI- R:>OWT03 H:A- JA45:ACER H71W.> J:HW.DI75J
